In [1]:
import os
import time
import argparse
import random
import numpy as np
import tensorflow as tf

from models.CTS import build_CTS
from configs.configs import Configs

from utils.read_data import read_essays, read_pos_vocab
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks

from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator

In [2]:
# seed = 12
# test_prompt_id = 1

# np.random.seed(seed)
# tf.random.set_seed(seed)
# random.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)
# print("Test prompt id is {} of type {}".format(test_prompt_id, type(test_prompt_id)))
# print("Seed: {}".format(seed))

# configs = Configs()
# data_path = configs.DATA_PATH
# train_path = data_path + str(test_prompt_id) + '/train.pk'
# dev_path = data_path + str(test_prompt_id) + '/dev.pk'
# test_path = data_path + str(test_prompt_id) + '/test.pk'
# features_path = configs.FEATURES_PATH
# readability_path = configs.READABILITY_PATH
# vocab_size = configs.VOCAB_SIZE
# epochs = configs.EPOCHS
# batch_size = configs.BATCH_SIZE

# read_configs = {
#         'train_path': train_path,
#         'dev_path': dev_path,
#         'test_path': test_path,
#         'features_path': features_path,
#         'readability_path': readability_path,
#         'vocab_size': vocab_size
#     }
# pos_vocab = read_pos_vocab(read_configs)
# train_data, dev_data, test_data = read_essays(read_configs, pos_vocab)

# max_sentlen = max(train_data['max_sentlen'], dev_data['max_sentlen'], test_data['max_sentlen'])
# max_sentnum = max(train_data['max_sentnum'], dev_data['max_sentnum'], test_data['max_sentnum'])
# print('max sent length: {}'.format(max_sentlen))
# print('max sent num: {}'.format(max_sentnum))
# train_data['y_scaled'] = get_scaled_down_scores(train_data['data_y'], train_data['prompt_ids'])
# dev_data['y_scaled'] = get_scaled_down_scores(dev_data['data_y'], dev_data['prompt_ids'])
# test_data['y_scaled'] = get_scaled_down_scores(test_data['data_y'], test_data['prompt_ids'])

# X_train_pos = pad_hierarchical_text_sequences(train_data['pos_x'], max_sentnum, max_sentlen)
# X_dev_pos = pad_hierarchical_text_sequences(dev_data['pos_x'], max_sentnum, max_sentlen)
# X_test_pos = pad_hierarchical_text_sequences(test_data['pos_x'], max_sentnum, max_sentlen)

# X_train_pos = X_train_pos.reshape((X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
# X_dev_pos = X_dev_pos.reshape((X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
# X_test_pos = X_test_pos.reshape((X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

# X_train_linguistic_features = np.array(train_data['features_x'])
# X_dev_linguistic_features = np.array(dev_data['features_x'])
# X_test_linguistic_features = np.array(test_data['features_x'])

# X_train_readability = np.array(train_data['readability_x'])
# X_dev_readability = np.array(dev_data['readability_x'])
# X_test_readability = np.array(test_data['readability_x'])

# Y_train = np.array(train_data['y_scaled'])
# Y_dev = np.array(dev_data['y_scaled'])
# Y_test = np.array(test_data['y_scaled'])

# X_train_attribute_rel = get_attribute_masks(Y_train)
# X_dev_attribute_rel = get_attribute_masks(Y_dev)
# X_test_attribute_rel = get_attribute_masks(Y_test)

# train_features_list = [X_train_pos, X_train_linguistic_features, X_train_readability]
# dev_features_list = [X_dev_pos, X_dev_linguistic_features, X_dev_readability]
# test_features_list = [X_test_pos, X_test_linguistic_features, X_test_readability]

In [3]:
new_array = np.loadtxt("samples.txt")

In [4]:
#model = build_CTS(len(pos_vocab), max_sentnum, max_sentlen,
#                      X_train_readability.shape[1],
#                      X_train_linguistic_features.shape[1],
#                      configs, Y_train.shape[1])
configs = Configs()

model = build_CTS(36, 97, 50, 35, 51, configs, 9)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pos_word_input (InputLayer)     [(None, 4850)]       0                                            
__________________________________________________________________________________________________
pos_x (Embedding)               (None, 4850, 50)     1800        pos_word_input[0][0]             
__________________________________________________________________________________________________
pos_x_maskedout (ZeroMaskedEntr (None, 4850, 50)     0           pos_x[0][0]                      
__________________________________________________________________________________________________
pos_drop_x (Dropout)            (None, 4850, 50)     0           pos_x_maskedout[0][0]            
_______________________________________________________________________________________

In [5]:
model.load_weights('saved_model/ckpt1')

In [6]:
def f(X):
    rows = [row for row in X]
    return model.predict([np.array(row[0:4850], dtype='int32').reshape(1,-1), 
                  np.array(row[4850:4901]).reshape(1,-1), 
                  np.array(row[4901:]).reshape(1,-1)]for row in rows)
    # return model.predict([np.array(row[0:4850], dtype='int32').reshape(1,-1), 
    #                       np.array(row[4850:4901]).reshape(1,-1), 
    #                       np.array(row[4901:]).reshape(1,-1)]).flatten()

In [7]:
# samples = []
# for i in range(0,10,1):
#     sample = np.concatenate((train_features_list[0][i],train_features_list[1][i],train_features_list[2][i]), axis=0)
#     samples.append(sample)
    
# samples = np.array(samples)

samples = new_array

In [8]:
import shap
    
#masker = shap.maskers.Text(custom_tokenizer)
explainer = shap.KernelExplainer(f, samples)

(10, 4936)


In [9]:
from IPython.display import Audio
sound_file = 'Done.mp3'
Audio(sound_file, autoplay=True)

In [10]:
shap_values = explainer.shap_values(samples[0])

[array([0]), array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9]), array([10]), array([11]), array([12]), array([13]), array([14]), array([15]), array([16]), array([17]), array([18]), array([19]), array([20]), array([21]), array([22]), array([23]), array([24]), array([25]), array([26]), array([27]), array([28]), array([29]), array([30]), array([31]), array([32]), array([33]), array([34]), array([35]), array([36]), array([37]), array([38]), array([39]), array([40]), array([41]), array([42]), array([43]), array([44]), array([45]), array([46]), array([47]), array([48]), array([49]), array([50]), array([51]), array([52]), array([53]), array([54]), array([55]), array([56]), array([57]), array([58]), array([59]), array([60]), array([61]), array([62]), array([63]), array([64]), array([65]), array([66]), array([67]), array([68]), array([69]), array([70]), array([71]), array([72]), array([73]), array([74]), array([75]), array([76]), array([7

In [16]:
np.savetxt("shap_values.txt",shap_values)

In [18]:
new_shap_values = np.loadtxt("shap_values.txt")

In [23]:
explainer.expected_value[0]

0.5806929618120193

In [25]:
len(new_shap_values[0])

4936

In [28]:
shap.force_plot(explainer.expected_value[0], new_shap_values[0], samples[0])

In [29]:
shap.force_plot(explainer.expected_value[1], new_shap_values[2], samples[0])